In [144]:
import re
import pandas as pd
import phonenumbers
from email_validator import validate_email, EmailNotValidError
import validators
import recordlinkage
from recordlinkage.preprocessing import phonetic, clean

In [145]:
# Local Env: nameparser is an external Python library not included in Snowflake Snowplow standard conda packages
from nameparser import HumanName

In [ ]:
# dbt Env
# Import wheel loader and nameparser library from stage: OTHER_PYTHON_PACKAGES
import wheel_loader
wheel_loader.load('nameparser-1.1.3-py2.py3-none-any.whl')
from nameparser import HumanName

In [146]:
# Local Env
# Create session (for dbt, this is passed into the model)
import json
import snowflake.connector
from snowflake.snowpark import Session

# Get secrets from file
secrets_file = open('secrets.json', 'r')
secrets = json.load(secrets_file)

# Connection parameters
snowflake_account = secrets.get('snowflake_account')
snowflake_user = secrets.get('snowflake_user')
snowflake_password = secrets.get('snowflake_password')
snowflake_role = secrets.get('snowflake_role')
snowflake_database = secrets.get('snowflake_database')
snowflake_warehouse = secrets.get('snowflake_warehouse')

schema_name = secrets.get('snowflake_schema')
connection_parameters = {
    "account": snowflake_account,
    "user": snowflake_user,
    "password": snowflake_password,
    "role": snowflake_role,
    "warehouse": snowflake_warehouse,
    "database": snowflake_database,
    "schema": schema_name
}
# Create a Snowflake Snowpark Session
session =  Session.builder.configs(connection_parameters).create()

# Create dbt object
class dbtObj:
    def __init__(self, config) -> None:
        self.config = config

dbt = dbtObj(
    config={
        # Variables passed from yaml file go here
        # 'pii_schema': 'COFFEE_SHOP',
    }
)

In [147]:
# Local Env: Get data from model int_contacts__union_all
contacts_query = """
        select *
        from {}.{}.int_contacts__union_all
""".format(snowflake_database, schema_name)

# Query database
contacts = session.sql(contacts_query).collect()
  
# Reset index
contacts_df = pd.DataFrame(contacts)
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,FILE_LAST_MODIFIED,FAVORITE_COLOR,COMPANY_REVENUE,COMPANY_EMPLOYEES,COMPANY_INDUSTRY,INTENT_SIGNALS,DO_NOT_CALL,IP_ADDRESS,COUNTRY_NAME,COUNTRY_CODE
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,2024-01-18 15:07:37,gray,NaN,NaN,None,None,None,None,None,None
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,2024-01-18 15:07:37,olive,NaN,NaN,None,None,None,None,None,None
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,2024-01-18 15:07:37,lime,NaN,NaN,None,None,None,None,None,None
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,2024-01-18 15:07:37,black,NaN,NaN,None,None,None,None,None,None
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,2024-01-18 15:07:37,navy,NaN,NaN,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,2024-01-18 14:48:35,None,576768431.0,4506.0,Tech,None,None,None,Chile,CL
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,2024-01-18 14:48:35,None,167947486.0,13578.0,Finance,None,None,None,Germany,DE
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,2024-01-18 14:48:35,None,992687764.0,2169.0,Healthcare,None,None,None,Spain,ES
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,2024-01-18 14:48:35,None,935436658.0,8717.0,Logistics,None,None,None,Japan,JP


In [ ]:
# dbt Env: Get data from model int_contacts__union_all
contacts_df = dbt.ref("int_contacts__union_all")

In [148]:
# Check Email: Uses Python library email-validator for validating and normalizing email addresses
# Reference: https://github.com/JoshData/python-email-validator
def check_email(email):
    if email is None:
        return None, None
    try:
        validator = validate_email(email, check_deliverability=False)
        email_info = validator.normalized
        is_valid = True
    except (EmailNotValidError, TypeError) as e:
        email_info = None
        is_valid = False
        print(f"INVALID email: {email}, ERROR: {e}")
    return is_valid, email_info


In [149]:
check_email("bogus@@")[0]

INVALID email: bogus@@, ERROR: The email address is not valid. It must have exactly one @-sign.


False

In [150]:
# Check Email: Add is_email_valid and email_info (normalized email or error message)
contacts_df['is_email_valid'] =  contacts_df['EMAIL_ADDRESS'].apply(check_email).str[0]
contacts_df['email_address_clean'] =  contacts_df['EMAIL_ADDRESS'].apply(check_email).str[1]

INVALID email: ms..dds@morales-fowler-jones.biz, ERROR: An email address cannot have two periods in a row.
INVALID email: mr..dds@reyes-ltd.xyz, ERROR: An email address cannot have two periods in a row.
INVALID email: dr..md@richardson-ltd.at, ERROR: An email address cannot have two periods in a row.
INVALID email: mr..bowman@barton-vega.io, ERROR: An email address cannot have two periods in a row.
INVALID email: mr..dudley@bailey-thomas.at, ERROR: An email address cannot have two periods in a row.
INVALID email: darryl.jr.@martinez-brown-manning.com, ERROR: An email address cannot have a period immediately before the @-sign.
INVALID email: mr..dds@howard-inc.xyz, ERROR: An email address cannot have two periods in a row.
INVALID email: mrs..obrien@dunn-lopez-jackson.ai, ERROR: An email address cannot have two periods in a row.
INVALID email: mjr.@case-ltd.org, ERROR: An email address cannot have a period immediately before the @-sign.
INVALID email: mrs..bennett@kelly-moses-pitts.com, 

In [151]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,COMPANY_REVENUE,COMPANY_EMPLOYEES,COMPANY_INDUSTRY,INTENT_SIGNALS,DO_NOT_CALL,IP_ADDRESS,COUNTRY_NAME,COUNTRY_CODE,is_email_valid,email_address_clean
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,NaN,NaN,None,None,None,None,None,None,True,michael.walker@ryan-soto-thompson.io
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,NaN,NaN,None,None,None,None,None,None,True,dkennedy@calderon-allen-graham.org
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,NaN,NaN,None,None,None,None,None,None,True,casey.miller@johnston-harris-brown.io
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,NaN,NaN,None,None,None,None,None,None,True,rhall@martin-morris.at
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,NaN,NaN,None,None,None,None,None,None,True,tsmith@white.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,576768431.0,4506.0,Tech,None,None,None,Chile,CL,True,satkinson@shaffer-baker-walsh.at
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,167947486.0,13578.0,Finance,None,None,None,Germany,DE,True,pbennett@brady-johnson-nelson.xyz
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,992687764.0,2169.0,Healthcare,None,None,None,Spain,ES,True,todd.adams@price-rodriguez-little.biz
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,935436658.0,8717.0,Logistics,None,None,None,Japan,JP,True,swallace@murray-riley-shields.at


In [152]:
# Check IP Address: Uses Python library validators for validating the IP Address
# Reference: https://validators.readthedocs.io/en/latest/#basic-validators
def check_ip_address(ip_address):
    if ip_address == '' or ip_address is None:
        return None
    is_valid = validators.ipv6(ip_address)
    if is_valid:
        # print('Valid ipv6')
        return True
    # print('Not ipv6')
    is_valid = validators.ipv4(ip_address)
    if is_valid:
        # print('Valid ipv4')
        return True
    # print('Not ipv4')
    print(f"INVALID ip_address: {ip_address}")
    return False


In [153]:
check_ip_address("255.255.255.1")

True

In [154]:
# Check IP Address: Add is_ip_addr_valid
contacts_df['is_ip_addr_valid'] =  contacts_df['IP_ADDRESS'].apply(check_ip_address)

In [155]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,COMPANY_EMPLOYEES,COMPANY_INDUSTRY,INTENT_SIGNALS,DO_NOT_CALL,IP_ADDRESS,COUNTRY_NAME,COUNTRY_CODE,is_email_valid,email_address_clean,is_ip_addr_valid
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,NaN,None,None,None,None,None,None,True,michael.walker@ryan-soto-thompson.io,None
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,NaN,None,None,None,None,None,None,True,dkennedy@calderon-allen-graham.org,None
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,NaN,None,None,None,None,None,None,True,casey.miller@johnston-harris-brown.io,None
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,NaN,None,None,None,None,None,None,True,rhall@martin-morris.at,None
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,NaN,None,None,None,None,None,None,True,tsmith@white.org,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,4506.0,Tech,None,None,None,Chile,CL,True,satkinson@shaffer-baker-walsh.at,None
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,13578.0,Finance,None,None,None,Germany,DE,True,pbennett@brady-johnson-nelson.xyz,None
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,2169.0,Healthcare,None,None,None,Spain,ES,True,todd.adams@price-rodriguez-little.biz,None
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,8717.0,Logistics,None,None,None,Japan,JP,True,swallace@murray-riley-shields.at,None


In [156]:
# Check Domain: Uses Python library validators for validating the Company Domain
# Reference: https://validators.readthedocs.io/en/latest/#basic-validators
def check_domain(company_domain):
    if company_domain == "" or company_domain is None:
        return None
    is_valid = validators.domain(company_domain)
    if is_valid:
        # print('Valid domain')
        return True    
    print(f"INVALID company_domain: {company_domain}")
    return False

In [157]:
check_domain("google.com")

True

In [158]:
# Check Company Domain: Add is_company_domain_valid
contacts_df['is_company_domain_valid'] =  contacts_df['COMPANY_DOMAIN'].apply(check_domain)

In [159]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,COMPANY_INDUSTRY,INTENT_SIGNALS,DO_NOT_CALL,IP_ADDRESS,COUNTRY_NAME,COUNTRY_CODE,is_email_valid,email_address_clean,is_ip_addr_valid,is_company_domain_valid
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,None,None,None,None,None,None,True,michael.walker@ryan-soto-thompson.io,None,True
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,None,None,None,None,None,None,True,dkennedy@calderon-allen-graham.org,None,True
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,None,None,None,None,None,None,True,casey.miller@johnston-harris-brown.io,None,True
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,None,None,None,None,None,None,True,rhall@martin-morris.at,None,True
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,None,None,None,None,None,None,True,tsmith@white.org,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,Tech,None,None,None,Chile,CL,True,satkinson@shaffer-baker-walsh.at,None,True
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,Finance,None,None,None,Germany,DE,True,pbennett@brady-johnson-nelson.xyz,None,True
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,Healthcare,None,None,None,Spain,ES,True,todd.adams@price-rodriguez-little.biz,None,True
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,Logistics,None,None,None,Japan,JP,True,swallace@murray-riley-shields.at,None,True


In [161]:
# Clean Phone Number: Validate and re-format phone number to E164 w/ extension
# Reference: https://github.com/daviddrysdale/python-phonenumbers
def clean_phone_number(phone, country_code='US'):
    if phone == "" or phone is None:
        return None, None
    # Replace leading 001- with +1-
    if phone[0:4] == '001-':# Clean Phone Numbers: Uses Python library phonenumbers for validating and formatting phone numbers
        phone_number = f"+1-{phone[4:]}"
    else:
        phone_number = phone
    try:
        phone_obj = phonenumbers.parse(phone_number, country_code)
        extension = phone_obj.extension
        formattted_phone_number = phonenumbers.format_number(phone_obj, phonenumbers.PhoneNumberFormat.E164)
        if extension:
            phone_str = f'{formattted_phone_number}x{extension}'
        else:
            phone_str = formattted_phone_number
        return True, phone_str
    except Exception as e:
        print(f"INVALID phone_number: {phone}, ERROR: {e}")
        return False, str(e)


In [162]:
phone_number = '+-3399999999-3333'
clean_phone_number(phone_number, 'US')


(True, '+33999999993333')

In [163]:
# Clean Phone Number: Validate and re-format phone number to E164 w/ extension
# This checks/cleans phone numbers based on 'US' country_code default, b/c only 1 source has country_code populated
# Also, all phone numbers in the datasets seem to follow the US/Canada phone format.
contacts_df['is_phone_us_valid'] =  contacts_df['PHONE_NUMBER'].apply(clean_phone_number).str[0]
contacts_df['phone_number_clean'] =  contacts_df['PHONE_NUMBER'].apply(clean_phone_number).str[1]

In [164]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,DO_NOT_CALL,IP_ADDRESS,COUNTRY_NAME,COUNTRY_CODE,is_email_valid,email_address_clean,is_ip_addr_valid,is_company_domain_valid,is_phone_us_valid,phone_number_clean
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,None,None,None,None,True,michael.walker@ryan-soto-thompson.io,None,True,True,+13667398129
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,None,None,None,None,True,dkennedy@calderon-allen-graham.org,None,True,True,+18912366451x6535
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,None,None,None,None,True,casey.miller@johnston-harris-brown.io,None,True,True,+12826654872
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,None,None,None,None,True,rhall@martin-morris.at,None,True,True,+17703066694
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,None,None,None,None,True,tsmith@white.org,None,True,True,+17178143111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,None,None,Chile,CL,True,satkinson@shaffer-baker-walsh.at,None,True,True,+14108266507
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,None,None,Germany,DE,True,pbennett@brady-johnson-nelson.xyz,None,True,True,+13133482696
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,None,None,Spain,ES,True,todd.adams@price-rodriguez-little.biz,None,True,True,+14543511516
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,None,None,Japan,JP,True,swallace@murray-riley-shields.at,None,True,True,+13757603115x286


In [203]:
# Parse Name: Returns first_name, last_name from a full_name that may contain initials, prefixes, suffixes.
# A better option would be to use an external library: nameparser
#   but this library is not available (in standard conda libraries) in Snowflake Snowpark
def parse_name(full_name):
    if full_name == "" or full_name is None:
        return None, None
    first_name = ''
    last_name = ''
    # Remove name prefix and suffix
    prefix = re.search('^((Mr|Mrs|Ms|Miss|Dr|Prof)(\.|\s)+)?', full_name).group()
    suffix = re.search('(\,|\.|\s){0,3}((Sr|Jr|I|II|III|IV|V|JD|MD|PhD)(\.|\s)*){0,3}$', full_name).group()
    if len(suffix) == 0:
        clean_full_name = full_name[len(prefix):].strip().title()
    else:
        clean_full_name = full_name[len(prefix):-len(suffix)].strip().title()
    name_parts = re.findall(r'\s|\,|\.|[^,\s]+', clean_full_name)
    # Deal with full_name: Last, First {M. I.}
    if ',' in name_parts:
        if len(name_parts) >= 2:
            last_name = name_parts[0].strip().title()
            get_index = name_parts.index(',') + 1
            for x in range(get_index, len(name_parts)):
                if len(name_parts[x].strip()) >= 1:
                    first_name = name_parts[x].strip().title()
        elif len(name_parts) == 1:
            first_name = ''
            last_name = name_parts[0].strip().title()
        else:
            first_name = ''
            last_name = ''
    # Deal with full_name: Fist {M. I.} Last
    else:
        if len(name_parts) >= 2:
            first_name = name_parts[0].strip().title()
            last_name = name_parts[-1].strip().title()
        elif len(name_parts) == 1:
            first_name = ''
            last_name = name_parts[-1].strip().title()
        else:
            first_name = ''
            last_name = ''
    # print(f"first_name: {first_name}, last_name: {last_name}")
    return first_name, last_name

In [204]:
parse_name('Prof. Jeffrey S. Huth, Jr. II PhD')

('Jeffrey', 'Huth')

In [167]:
parse_name('Mr. Huth, Jeffrey')

('Jeffrey', 'Huth')

In [168]:
# Parse name into first and last name for matching using nameparser library.
#   nameparser is an external Python library not included in Snowflake Snowplow standard conda packages
# Reference: https://github.com/derek73/python-nameparser
def parse_name_2(full_name):
    if full_name == '' or full_name is None:
        return '', ''
    first_name = ''
    last_name = ''
    try:
        name_obj = HumanName(full_name)
        last_name = name_obj.last
        first_name = name_obj.first
    except Exception as e:
        print(f"Error: {e}")
    return first_name, last_name

In [26]:
parse_name_2("Dr. Jeffrey S. Huth, Jr. PhD")

('Jeffrey', 'Huth')

In [169]:
parse_name_2("Dr. Juan Q. Xavier de la Vega III (Doc Vega)")

('Juan', 'de la Vega')

In [170]:
# Add first_name and last_name to the dataset
contacts_df['first_name'] =  contacts_df['NAME'].apply(parse_name_2).str[0]
contacts_df['last_name'] =  contacts_df['NAME'].apply(parse_name_2).str[1]

In [171]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,COUNTRY_NAME,COUNTRY_CODE,is_email_valid,email_address_clean,is_ip_addr_valid,is_company_domain_valid,is_phone_us_valid,phone_number_clean,first_name,last_name
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,None,None,True,michael.walker@ryan-soto-thompson.io,None,True,True,+13667398129,Michael,Walker
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,None,None,True,dkennedy@calderon-allen-graham.org,None,True,True,+18912366451x6535,Douglas,Kennedy
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,None,None,True,casey.miller@johnston-harris-brown.io,None,True,True,+12826654872,Casey,Miller
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,None,None,True,rhall@martin-morris.at,None,True,True,+17703066694,Ronald,Hall
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,None,None,True,tsmith@white.org,None,True,True,+17178143111,Troy,Smith
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,Chile,CL,True,satkinson@shaffer-baker-walsh.at,None,True,True,+14108266507,Steven,Atkinson
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,Germany,DE,True,pbennett@brady-johnson-nelson.xyz,None,True,True,+13133482696,Peter,Bennett
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,Spain,ES,True,todd.adams@price-rodriguez-little.biz,None,True,True,+14543511516,Todd,Adams
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,Japan,JP,True,swallace@murray-riley-shields.at,None,True,True,+13757603115x286,Sean,Wallace


In [173]:
# Pre-process names to clean and get phonetic versions
# Clean the first, last, company names and add phonetic versions metaphone, match_rating, and nysiis
contacts_df['first_name_clean'] =  clean(contacts_df['first_name'])
contacts_df['last_name_clean'] =  clean(contacts_df['last_name'])
contacts_df['company_name_clean'] =  clean(contacts_df['COMPANY_NAME'])

# Metaphone Approach, Reference: https://en.wikipedia.org/wiki/Metaphone
contacts_df['first_name_mp'] =  phonetic(contacts_df['first_name_clean'], 'match_rating')
contacts_df['last_name_mp'] =  phonetic(contacts_df['last_name_clean'], 'match_rating')
contacts_df['company_name_mp'] =  phonetic(contacts_df['company_name_clean'], 'match_rating')

# Match Rating Approach, Reference: https://en.wikipedia.org/wiki/Match_rating_approach
contacts_df['first_name_mra'] =  phonetic(contacts_df['first_name_clean'], 'match_rating')
contacts_df['last_name_mra'] =  phonetic(contacts_df['last_name_clean'], 'match_rating')
contacts_df['company_name_mra'] =  phonetic(contacts_df['company_name_clean'], 'match_rating')

# NYSIIS Approach, Reference: https://en.wikipedia.org/wiki/New_York_State_Identification_and_Intelligence_System
contacts_df['first_name_nysiis'] =  phonetic(contacts_df['first_name_clean'], 'nysiis')
contacts_df['last_name_nysiis'] =  phonetic(contacts_df['last_name_clean'], 'nysiis')
contacts_df['company_name_nysiis'] =  phonetic(contacts_df['company_name_clean'], 'nysiis')

In [174]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,company_name_clean,first_name_mp,last_name_mp,company_name_mp,first_name_mra,last_name_mra,company_name_mra,first_name_nysiis,last_name_nysiis,company_name_nysiis
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,ryan soto and thompson,MCHL,WLKR,RYNPSN,MCHL,WLKR,RYNPSN,MACAL,WALCAR,RYANSATANDTANPSAN
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,calderon allen and graham,DGLS,KNDY,CLDRHM,DGLS,KNDY,CLDRHM,DAGL,CANADY,CALDARANALANANDGRAHAN
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,johnston harris and brown,CSY,MLR,JHNRWN,CSY,MLR,JHNRWN,CASY,MALAR,JANSTANARASANDBRAON
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,martin morris,RNLD,HL,MRTMRS,RNLD,HL,MRTMRS,RANALD,HAL,MARTANAR
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,white llc,TRY,SMTH,WHTLC,TRY,SMTH,WHTLC,TRY,SNAT,WATALC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,shaffer baker and walsh,STVN,ATKNSN,SHFLSH,STVN,ATKNSN,SHFLSH,STAFAN,ATCANSAN,SAFARBACARANDWAL
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,brady johnson and nelson,PTR,BNT,BRDLSN,PTR,BNT,BRDLSN,PATAR,BANAT,BRADYJANSANANDNALSAN
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,price rodriguez and little,TD,ADMS,PRCLTL,TD,ADMS,PRCLTL,TAD,ADAN,PRACARADRAGASANDLATL
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,murray riley and shields,SN,WLC,MRYLDS,SN,WLC,MRYLDS,SAN,WALAC,MARAYRALAYANDSALD


In [175]:
# Get First Initial: Substring first letter of first name
def get_first_initial(first_name):
    if first_name == '' or first_name is None:
        return None
    return first_name[0]

In [176]:
get_first_initial('Jeff')

'J'

In [177]:
# Get First Initial: Add a column for first_initial
contacts_df['first_initial'] =  contacts_df['first_name'].apply(get_first_initial)

In [178]:
display(contacts_df)

,SOURCE_CONTACT_KEY,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,...,first_name_mp,last_name_mp,company_name_mp,first_name_mra,last_name_mra,company_name_mra,first_name_nysiis,last_name_nysiis,company_name_nysiis,first_initial
0,62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,...,MCHL,WLKR,RYNPSN,MCHL,WLKR,RYNPSN,MACAL,WALCAR,RYANSATANDTANPSAN,M
1,2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,...,DGLS,KNDY,CLDRHM,DGLS,KNDY,CLDRHM,DAGL,CANADY,CALDARANALANANDGRAHAN,D
2,10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,...,CSY,MLR,JHNRWN,CSY,MLR,JHNRWN,CASY,MALAR,JANSTANARASANDBRAON,C
3,1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,...,RNLD,HL,MRTMRS,RNLD,HL,MRTMRS,RANALD,HAL,MARTANAR,R
4,351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,...,TRY,SMTH,WHTLC,TRY,SMTH,WHTLC,TRY,SNAT,WATALC,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,...,STVN,ATKNSN,SHFLSH,STVN,ATKNSN,SHFLSH,STAFAN,ATCANSAN,SAFARBACARANDWAL,S
2558,51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,...,PTR,BNT,BRDLSN,PTR,BNT,BRDLSN,PATAR,BANAT,BRADYJANSANANDNALSAN,P
2559,d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,...,TD,ADMS,PRCLTL,TD,ADMS,PRCLTL,TAD,ADAN,PRACARADRAGASANDLATL,T
2560,672f1bae98aecbbd17469698283b976c,acme,Sean Wallace,swallace@murray-riley-shields.at,001-375-760-3115x286,Team Lead,"Murray, Riley and Shields",murray-riley-shields.at,2023-12-29,2023-12-29,...,SN,WLC,MRYLDS,SN,WLC,MRYLDS,SAN,WALAC,MARAYRALAYANDSALD,S


In [179]:
# Set unique index on contacts_df
contacts_df = contacts_df.set_index('SOURCE_CONTACT_KEY')


In [180]:
# Sort the dataframme
contacts_df.sort_values('last_name').head()
display(contacts_df)

,SOURCE_NAME,NAME,EMAIL_ADDRESS,PHONE_NUMBER,TITLE,COMPANY_NAME,COMPANY_DOMAIN,CREATED_AT,UPDATED_AT,FILE_NM,...,first_name_mp,last_name_mp,company_name_mp,first_name_mra,last_name_mra,company_name_mra,first_name_nysiis,last_name_nysiis,company_name_nysiis,first_initial
SOURCE_CONTACT_KEY,,,,,,,,,,,,,,,,,,,,,
62afe6edd2e1b38285768d566398716f,crm,Michael Walker,michael.walker@ryan-soto-thompson.io,366-739-8129,IC,"Ryan, Soto and Thompson",ryan-soto-thompson.io,2023-01-01,2023-03-02,contacts/crm__contacts.csv,...,MCHL,WLKR,RYNPSN,MCHL,WLKR,RYNPSN,MACAL,WALCAR,RYANSATANDTANPSAN,M
2e0ab3bebb2cb59f274c7a8d3242f30a,crm,Douglas Kennedy,dkennedy@calderon-allen-graham.org,001-891-236-6451x6535,IC,"Calderon, Allen and Graham",calderon-allen-graham.org,2023-01-01,2023-11-27,contacts/crm__contacts.csv,...,DGLS,KNDY,CLDRHM,DGLS,KNDY,CLDRHM,DAGL,CANADY,CALDARANALANANDGRAHAN,D
10bccf1772b0871e8ffc3b6fe71e028a,crm,Casey Miller,casey.miller@johnston-harris-brown.io,001-282-665-4872,IC,"Johnston, Harris and Brown",johnston-harris-brown.io,2023-01-01,2023-10-26,contacts/crm__contacts.csv,...,CSY,MLR,JHNRWN,CSY,MLR,JHNRWN,CASY,MALAR,JANSTANARASANDBRAON,C
1b4b88791adea115155b95c3087450e6,crm,Ronald Hall,rhall@martin-morris.at,(770)306-6694,CEO (Chief Executive Officer),Martin-Morris,martin-morris.at,2023-01-01,2023-12-18,contacts/crm__contacts.csv,...,RNLD,HL,MRTMRS,RNLD,HL,MRTMRS,RANALD,HAL,MARTANAR,R
351cbda9179109c81f7913343916bfcb,crm,Troy Smith,tsmith@white.org,+1-717-814-3111,IC,White LLC,white.org,2023-01-01,2023-09-25,contacts/crm__contacts.csv,...,TRY,SMTH,WHTLC,TRY,SMTH,WHTLC,TRY,SNAT,WATALC,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4a301b7835136852512017c195e5d57a,acme,Steven Atkinson,satkinson@shaffer-baker-walsh.at,410.826.6507,IC,"Shaffer, Baker and Walsh",shaffer-baker-walsh.at,2023-12-22,2023-12-22,contacts/acme__contacts.csv,...,STVN,ATKNSN,SHFLSH,STVN,ATKNSN,SHFLSH,STAFAN,ATCANSAN,SAFARBACARANDWAL,S
51b6c194ee7f54a06022016581b73671,acme,Peter Bennett,pbennett@brady-johnson-nelson.xyz,3133482696,IC,"Brady, Johnson and Nelson",brady-johnson-nelson.xyz,2023-12-25,2023-12-25,contacts/acme__contacts.csv,...,PTR,BNT,BRDLSN,PTR,BNT,BRDLSN,PATAR,BANAT,BRADYJANSANANDNALSAN,P
d121ed3c7f55c56ac8d0fa13a6eba6ee,acme,Todd Adams,todd.adams@price-rodriguez-little.biz,454-351-1516,IC,"Price, Rodriguez and Little",price-rodriguez-little.biz,2023-12-28,2023-12-28,contacts/acme__contacts.csv,...,TD,ADMS,PRCLTL,TD,ADMS,PRCLTL,TAD,ADAN,PRACARADRAGASANDLATL,T


In [181]:
indexer = recordlinkage.Index()
indexer.full()
candidate_record_pairs = indexer.index(contacts_df)
print("Number of candidate record pairs", len(candidate_record_pairs))
#candidate_record_pairs.to_frame(index=False)
#(5000*5000-5000)/2 = 12 497 500


# This is WAY TOO MANY!

Number of candidate record pairs 3280641


In [182]:
# Match and de-duplicate with the recordlinkage Python library
# Reference: https://recordlinkage.readthedocs.io/en/latest/guides/data_deduplication.html

# Initialize indexer: Only compare records with the same last_name_clean
indexer_1 = recordlinkage.Index()
indexer_1.block(left_on="last_name_clean")
candidate_links_1 = indexer_1.index(contacts_df)

# Initialize indexer: Only compare records with the same phone_number_clean
indexer_2 = recordlinkage.Index()
indexer_2.block(left_on="phone_number_clean")
candidate_links_2 = indexer_2.index(contacts_df)

# Every record against every record
indexer_3 = recordlinkage.Index()
indexer_3.full()
candidate_links_3 = indexer_3.index(contacts_df)

In [183]:
# Comparison types for fields in all source files
compare_cl = recordlinkage.Compare()
compare_cl.exact("last_name_mp", "last_name_mp", label="last_name_mp")
compare_cl.exact("first_name_mp", "first_name_mp", label="first_name_mp")
compare_cl.exact("first_initial", "first_initial", label="first_initial")
compare_cl.exact("email_address_clean", "email_address_clean", label="email_address_clean")
compare_cl.exact("phone_number_clean", "phone_number_clean", label="phone_number_clean")
compare_cl.exact("TITLE", "TITLE", label="title")
compare_cl.string("company_name_clean", "company_name_clean", method="jarowinkler", threshold=0.7, label="company_name_clean")

features_1 = compare_cl.compute(candidate_links_1, contacts_df)
features_2 = compare_cl.compute(candidate_links_2, contacts_df)
features_3 = compare_cl.compute(candidate_links_3, contacts_df)

In [184]:
features_3.describe()

,last_name_mp,first_name_mp,first_initial,email_address_clean,phone_number_clean,title,company_name_clean
count,3.280641e+06,3.280641e+06,3.280641e+06,3.280641e+06,3.280641e+06,3.280641e+06,3.280641e+06
mean,4.900872e-03,7.013264e-03,7.685480e-02,5.047794e-04,6.449959e-04,5.245234e-01,1.822784e-02
std,6.983448e-02,8.345106e-02,2.663610e-01,2.246163e-02,2.538858e-02,4.993983e-01,1.337744e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [185]:
# Matches based on the number of matching features
features_3.sum(axis=1).value_counts().sort_index(ascending=False)

7.0       1345
6.0        326
5.0        609
4.0        529
3.0      13320
2.0     168362
1.0    1682347
0.0    1413803
Name: count, dtype: int64

In [186]:
# Create dataframes of matches based on each feature set and the required number of matching features
matches_1 = features_1[features_1.sum(axis=1) >= 4]
matches_2 = features_2[features_2.sum(axis=1) >= 4]
matches_3 = features_3[features_3.sum(axis=1) >= 5]
matches_3

last_name_mp  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                             
64db62708f598d0b7119d6f7e85506ef 266454807083e33b371c27f1b791419f             1   
6f88a3025b514be885813d27e44d0e73 e7f6d398e665d8878e0fa8bbf5c04b13             1   
1d6544eaafd5ee8bb49f9b273d2e71a6 f72d4dfbbc40a810c7f973aca6245888             1   
363f7e91fabf50c08d20432a139d567b 5ae3867f7fef687bc5fdcffa0fd25d67             1   
294284367f383ee2c4cee1c75807aab2 93e861690de01a286036483a37f66a76             1   
...                                                                         ...   
d121ed3c7f55c56ac8d0fa13a6eba6ee 9cb54cff1a5a6a72c16ceeeb9154894d             1   
672f1bae98aecbbd17469698283b976c bb11059c7febc5d65edb38fbca930365             1   
                                 19a9297f2ebec21121290668ab89109e             1   
3972d538bf202fb438a0ac95e054367d 668b7c07fa1d8b51c2bf650530893a6d             1   
                                 d14aee7bce4016cd67755a6c356a9de9             1   

                                                                   first_name_mp  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                              
64db62708f598d0b7119d6f7e85506ef 266454807083e33b371c27f1b791419f              1   
6f88a3025b514be885813d27e44d0e73 e7f6d398e665d8878e0fa8bbf5c04b13              1   
1d6544eaafd5ee8bb49f9b273d2e71a6 f72d4dfbbc40a810c7f973aca6245888              1   
363f7e91fabf50c08d20432a139d567b 5ae3867f7fef687bc5fdcffa0fd25d67              1   
294284367f383ee2c4cee1c75807aab2 93e861690de01a286036483a37f66a76              1   
...                                                                          ...   
d121ed3c7f55c56ac8d0fa13a6eba6ee 9cb54cff1a5a6a72c16ceeeb9154894d              1   
672f1bae98aecbbd17469698283b976c bb11059c7febc5d65edb38fbca930365              1   
                                 19a9297f2ebec21121290668ab89109e              1   
3972d538bf202fb438a0ac95e054367d 668b7c07fa1d8b51c2bf650530893a6d              1   
                                 d14aee7bce4016cd67755a6c356a9de9              1   

                                                                   first_initial  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                              
64db62708f598d0b7119d6f7e85506ef 266454807083e33b371c27f1b791419f              1   
6f88a3025b514be885813d27e44d0e73 e7f6d398e665d8878e0fa8bbf5c04b13              1   
1d6544eaafd5ee8bb49f9b273d2e71a6 f72d4dfbbc40a810c7f973aca6245888              1   
363f7e91fabf50c08d20432a139d567b 5ae3867f7fef687bc5fdcffa0fd25d67              1   
294284367f383ee2c4cee1c75807aab2 93e861690de01a286036483a37f66a76              1   
...                                                                          ...   
d121ed3c7f55c56ac8d0fa13a6eba6ee 9cb54cff1a5a6a72c16ceeeb9154894d              1   
672f1bae98aecbbd17469698283b976c bb11059c7febc5d65edb38fbca930365              1   
                                 19a9297f2ebec21121290668ab89109e              1   
3972d538bf202fb438a0ac95e054367d 668b7c07fa1d8b51c2bf650530893a6d              1   
                                 d14aee7bce4016cd67755a6c356a9de9              1   

                                                                   email_address_clean  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                                    
64db62708f598d0b7119d6f7e85506ef 266454807083e33b371c27f1b791419f                    0   
6f88a3025b514be885813d27e44d0e73 e7f6d398e665d8878e0fa8bbf5c04b13                    0   
1d6544eaafd5ee8bb49f9b273d2e71a6 f72d4dfbbc40a810c7f973aca6245888                    0   
363f7e91fabf50c08d20432a139d567b 5ae3867f7fef687bc5fdcffa0fd25d67                    0   
294284367f383ee2c4cee1c75807aab2 93e861690de01a286036483a37f66a76                    0   
...                                                                                ...   
d121ed3c7f55c56ac8d0fa13a6eba6ee 9cb54cff1a5a6a72c16ceeeb9154894d                

In [81]:
matches_1

,,last_name_mp,first_name_mp,first_initial,email_address_clean,phone_number_clean,title,company_name_clean
SOURCE_CONTACT_KEY_1,SOURCE_CONTACT_KEY_2,,,,,,,
4473b2e298a14b96e070bdd35931cdac,48be23db1ce587990b38ab791b5894b6,1,1,1,0,1,1,0.0
5adbfe50986b2605164ec839d8b6b043,6a4838bc3c363453ec82617e275529a1,1,1,1,0,1,1,0.0
6f81b478116852a0fd157d14ed2da76c,cd8ad00270d1bb1f8f733f33ad004a2f,1,1,1,0,1,1,0.0
46aafbdac1a8ae0bd731800725989d35,5235dc3fb1d8d60afa2a6ecb2e34e0b4,1,1,1,0,1,1,0.0
245d74886b273b6cd8cfbdf110e5cc3b,6a4838bc3c363453ec82617e275529a1,1,1,1,0,1,0,0.0
...,...,...,...,...,...,...,...,...
01d1103f0f43a81d7dc999b95686d45d,420703d515811d3e06a8dc889298a440,1,1,1,0,1,1,0.0
7de3e72ed60fb434368e356b567182ff,f832cae619aed2b79a0d6e553be07be9,1,1,1,0,1,1,0.0
50eb879a2dd2ee57917906fbffb2302f,f832cae619aed2b79a0d6e553be07be9,1,1,1,0,1,1,0.0


In [99]:
len(matches_1)

266

In [100]:
len(matches_2)

197

In [261]:
len(matches_3)

2280

In [224]:
# Create and squash distinct sets of matched ids
comnbined_matches = pd.concat([matches_1, matches_2, matches_3]).sort_values(by = ['SOURCE_CONTACT_KEY_1', 'SOURCE_CONTACT_KEY_2'])
comnbined_matches

last_name_mp  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                             
002478c5c039af2573e0eb4ca4b2dd39 6cd6b6b9207ed30567a5b616331a4f24             1   
                                 6cd6b6b9207ed30567a5b616331a4f24             1   
                                 a159962b278fad8e492b8e13b9922b5e             1   
005fe2f9e3803d6d049e6cd2dae8583f 0c1568d48c39346ad4573b7ab47ac4c4             1   
                                 0c1568d48c39346ad4573b7ab47ac4c4             1   
...                                                                         ...   
ffe87a08cf2ca9301af709e1dd50e1e3 58a0970f09ef35de0e6a632640816321             1   
                                 b1f633c4e0c47e1afadb98b2b0bc0786             1   
fff8c7014917f9571aaebcede32059be 7ea9890c5e0efff6e5b8d4979ea6af83             1   
                                 7ea9890c5e0efff6e5b8d4979ea6af83             1   
                                 7ea9890c5e0efff6e5b8d4979ea6af83             1   

                                                                   first_name_mp  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                              
002478c5c039af2573e0eb4ca4b2dd39 6cd6b6b9207ed30567a5b616331a4f24              1   
                                 6cd6b6b9207ed30567a5b616331a4f24              1   
                                 a159962b278fad8e492b8e13b9922b5e              1   
005fe2f9e3803d6d049e6cd2dae8583f 0c1568d48c39346ad4573b7ab47ac4c4              1   
                                 0c1568d48c39346ad4573b7ab47ac4c4              1   
...                                                                          ...   
ffe87a08cf2ca9301af709e1dd50e1e3 58a0970f09ef35de0e6a632640816321              1   
                                 b1f633c4e0c47e1afadb98b2b0bc0786              1   
fff8c7014917f9571aaebcede32059be 7ea9890c5e0efff6e5b8d4979ea6af83              1   
                                 7ea9890c5e0efff6e5b8d4979ea6af83              1   
                                 7ea9890c5e0efff6e5b8d4979ea6af83              1   

                                                                   first_initial  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                              
002478c5c039af2573e0eb4ca4b2dd39 6cd6b6b9207ed30567a5b616331a4f24              1   
                                 6cd6b6b9207ed30567a5b616331a4f24              1   
                                 a159962b278fad8e492b8e13b9922b5e              1   
005fe2f9e3803d6d049e6cd2dae8583f 0c1568d48c39346ad4573b7ab47ac4c4              1   
                                 0c1568d48c39346ad4573b7ab47ac4c4              1   
...                                                                          ...   
ffe87a08cf2ca9301af709e1dd50e1e3 58a0970f09ef35de0e6a632640816321              1   
                                 b1f633c4e0c47e1afadb98b2b0bc0786              1   
fff8c7014917f9571aaebcede32059be 7ea9890c5e0efff6e5b8d4979ea6af83              1   
                                 7ea9890c5e0efff6e5b8d4979ea6af83              1   
                                 7ea9890c5e0efff6e5b8d4979ea6af83              1   

                                                                   email_address_clean  \
SOURCE_CONTACT_KEY_1             SOURCE_CONTACT_KEY_2                                    
002478c5c039af2573e0eb4ca4b2dd39 6cd6b6b9207ed30567a5b616331a4f24                    1   
                                 6cd6b6b9207ed30567a5b616331a4f24                    1   
                                 a159962b278fad8e492b8e13b9922b5e                    0   
005fe2f9e3803d6d049e6cd2dae8583f 0c1568d48c39346ad4573b7ab47ac4c4                    1   
                                 0c1568d48c39346ad4573b7ab47ac4c4                    1   
...                                                                                ...   
ffe87a08cf2ca9301af709e1dd50e1e3 58a0970f09ef35de0e6a632640816321                

In [260]:
key_maps = []
# Get all key maps
for idx, row in comnbined_matches.iterrows():
    keys = [idx[0], idx[1]]
    key_maps.append(keys)
key_maps.sort(key=lambda x: [x[0], x[1]])
len(key_maps)  # Too many, need to squash into distict matching group sets

6946

In [257]:
i = 0
groups = []

for km in key_maps:
    in_group = False
    if i == 0:
        groups.append(km)
    else:
        for grp in groups:
            if km[0] in grp and km[1] not in grp:
                grp.append(km[1])
                in_group = True
            elif km[1] in grp and km[0] not in grp:
                grp.append(km[0])
                in_group = True
            elif km[1] in grp and km[0] in grp:
                in_group = True
    if not(in_group):
        groups.append(km)
    i = i + 1

len(groups)

937

In [258]:
# Ensure each group is a unique, sorted set of keys
new_groups = []
for grp in groups:
    if len(grp) > 0:
        new_grp = sorted(list(set(grp)))
        new_groups.append(new_grp)

len(new_groups)

937

In [259]:
new_df = pd.DataFrame({"matches": new_groups})
display(new_df)

,matches
0,"[002478c5c039af2573e0eb4ca4b2dd39, 6cd6b6b9207..."
1,"[002478c5c039af2573e0eb4ca4b2dd39, 6cd6b6b9207..."
2,"[005fe2f9e3803d6d049e6cd2dae8583f, 0c1568d48c3..."
3,"[006422b9502c4dd2a60017dfe79fabfd, 15b889e6478..."
4,"[0074dc1ac23c22f2251e218059bab936, 51659802171..."
...,...
932,"[27b4b763a70cab13ad545de5a6d3e773, fd63a220601..."
933,"[bc3b01575b81c55e68c109637de86535, fe49585d5ab..."
934,"[1aa32777ba75e0690d387a518434d446, fe7eb6008b2..."
935,"[2140b08bccf1cae9b3beee4e6911bbed, ff171c5c13b..."


In [190]:
new_df.to_csv('matches.csv')